<h1><center>PCA for Identifying Key Variables in Donor Funding Analysis<center><h1>


<img src="files/images/logo.svg">

### 1. Introduction

#### a).Objective
The objective of this task is to develop a model to address the growing number of referrals at the Anderson Cancer Center. The model's first step is to identify essential variables for securing donor funding. Principal Component Analysis (PCA) has been chosen as the most suitable technique for this task. The following tasks will be implemented:
- PCA Implementation:

Utilize PCA to demonstrate how essential variables can be acquired from the cancer dataset available from sklearn.datasets.

- Dimensionality Reduction:

Reduce the dataset into 2 PCA components for the project.

- Implement logistic regression for prediction.

Implement a logistic regression for a defined prediction on the dataset.

#### b). Background
Over the past few months, the number of referral cases at the Anderson Cancer Center has been growing rapidly. In order to provide a comprehensive and integrated health service for the people,the center has identified the need for using data to diagnose this growth, and to communicate the findings to simulate donor funding. This study aims to develop a model that will aid the center in addressing the growing number of referrals and identify the key variables that will appeal to donor funding.

#### c). An Overview of PCA
Principal Component Analysis (PCA) is a dimensionality reduction technique that is used to reduce the number of dimensions in a large dataset while retaining the original information. The "Curse of Dimensionality" is a common problem in machine learning. Many dimensions in a dataset often lead to overfitting (where a model fits too closely) and or multicollinearity (where variables are related, affecting model performance). PCA irons out these problems by extracting the most informative features from large datasets while preserving the most relevant information from the initial dataset (https://www.ibm.com/topics/principal-component-analysis).

This statistical technique involves both linear algebra and matrix operations, and it transforms the original dataset into a new coordinate system that is structured by the principal components. The eigenvectors and eigenvalues from the covariance matrix that underpin the principal components allow for the analysis of these linear transformations.
<img src="files/images/pca.png">
Figure: Illustration of PCA. (Source: IBM)

### 2. Data Exploration
We will 

### 3. Data Cleaning and Preprocessing

### 4. Exploratory Data Analysis (EDA)

### 5. Principal Component Analysis (PCA)

### 6. Visualizing Results

### 7. Interpretation and Key Variable Identification

### 8. Conclusion and Next Steps